![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DistilBertForMultipleChoice.ipynb)

## Import ONNX DistilBertForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DistilBertForMultipleChoice` is only available since in `Spark NLP 5.6.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for question answering via `DistilBertForMultipleChoice` or `TFDistilBertForMultipleChoice`. These models are usually under `Multiple Choice` category and have `bert` in their labels
- Reference: [DistilBertForMultipleChoice](https://huggingface.co/docs/transformers/main/en/model_doc/distilbert#transformers.DistilBertForMultipleChoice)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.

In [ ]:
!pip install -q --upgrade transformers[onnx] optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use the treained model above as an example and load it as a `ORTModelForMultipleChoice`, representing an ONNX model.

In [ ]:
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "daze-unlv/distilbert-distilbert-base-uncased"
EXPORT_PATH = f"onnx_models/distilbert_multiple_choice"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -lh {EXPORT_PATH}

total 257M
-rw-r--r-- 1 root root  579 Dec 31 23:04 config.json
-rw-r--r-- 1 root root 256M Dec 31 23:04 model.onnx
-rw-r--r-- 1 root root  695 Dec 31 23:04 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K Dec 31 23:04 tokenizer_config.json
-rw-r--r-- 1 root root 695K Dec 31 23:04 tokenizer.json
-rw-r--r-- 1 root root 227K Dec 31 23:04 vocab.txt


We are missing the `vocab.txt` from the tokenizer. Let's save the tokenizer and move this file to `assets` folder which Spark NLP will look for

In [ ]:
!mkdir {EXPORT_PATH}/assets

In [ ]:
!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

Voila! We have our vocab.txt inside assets directory

In [ ]:
!ls -lR {EXPORT_PATH}

onnx_models/distilbert_multiple_choice:
total 262392
drwxr-xr-x 2 root root      4096 Dec 31 23:04 assets
-rw-r--r-- 1 root root       579 Dec 31 23:04 config.json
-rw-r--r-- 1 root root 267957763 Dec 31 23:04 model.onnx
-rw-r--r-- 1 root root       695 Dec 31 23:04 special_tokens_map.json
-rw-r--r-- 1 root root      1254 Dec 31 23:04 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Dec 31 23:04 tokenizer.json

onnx_models/distilbert_multiple_choice/assets:
total 228
-rw-r--r-- 1 root root 231508 Dec 31 23:04 vocab.txt


## Import and Save DistilBertForMultipleChoice in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
!wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

Apache Spark version: 3.5.3


- Let's use `loadSavedModel` functon in `DistilBertForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForMultipleChoice` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *


distilBertMultpleChoiceClassifier = DistilBertForMultipleChoice.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

Let's save it on disk so it is easier to be moved around and also be used later via .load function

In [ ]:
distilBertMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `DistilBertForMultipleChoice` model in Spark NLP 🚀 pipeline!

In [ ]:
 testing_data = [
            ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.",
             "It is eaten with a fork and a knife, It is eaten while held in the hand."),

            ("The Eiffel Tower is located in which country?",
             "Germany, France, Italy"),

            ("Which animal is known as the king of the jungle?",
             "Lion, Elephant, Tiger, Leopard"),

            ("Water boils at what temperature?",
             "90°C, 120°C, 100°C"),

            ("Which planet is known as the Red Planet?",
             "Jupiter, Mars, Venus"),

            ("Which language is primarily spoken in Brazil?",
             "Spanish, Portuguese, English"),

            ("The Great Wall of China was built to protect against invasions from which group?",
             "The Greeks, The Romans, The Mongols, The Persians"),

            ("Which chemical element has the symbol 'O'?",
             "Oxygenm, Osmium, Ozone"),

            ("Which continent is the Sahara Desert located in?",
             "Asia, Africa, South America"),

            ("Which artist painted the Mona Lisa?",
             "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
        ]

In [ ]:
testing_df = spark.createDataFrame(testing_data, ["question", "choices"])
testing_df.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

In [ ]:
document_assembler = MultiDocumentAssembler() \
            .setInputCols(["question", "choices"]) \
            .setOutputCols(["document_question", "document_choices"])

distilbert_for_multiple_choice = DistilBertForMultipleChoice() \
    .load("./{}_spark_nlp_onnx".format(MODEL_NAME)) \
    .setInputCols(["document_question", "document_choices"])\
    .setOutputCol("answer") \
    .setBatchSize(4)

pipeline = Pipeline(stages=[document_assembler, distilbert_for_multiple_choice])
pipeline_model = pipeline.fit(testing_df)

pipeline_df = pipeline_model.transform(testing_df)

pipeline_df.select("answer").show(truncate=False)

+----------------------------------------------------------------------------------------------------------+
|answer                                                                                                    |
+----------------------------------------------------------------------------------------------------------+
|[{chunk, 0, 34, It is eaten with a fork and a knife, {sentence -> 0, chunk -> 0, score -> 0.6048505}, []}]|
|[{chunk, 0, 6, Germany, {sentence -> 0, chunk -> 0, score -> 0.39137164}, []}]                            |
|[{chunk, 0, 5,  Tiger, {sentence -> 0, chunk -> 0, score -> 0.2897997}, []}]                              |
|[{chunk, 0, 3, 90°C, {sentence -> 0, chunk -> 0, score -> 0.35916787}, []}]                               |
|[{chunk, 0, 6, Jupiter, {sentence -> 0, chunk -> 0, score -> 0.35939977}, []}]                            |
|[{chunk, 0, 7,  English, {sentence -> 0, chunk -> 0, score -> 0.3640033}, []}]                            |
|[{chunk, 0, 11,  T